### Decorators

In this notebook we’ll:

- Build the basic decorator pattern
- Use it to log calls
- Improve it with `functools.wraps`
- Create **parameterized decorators** (decorators with arguments)
- Stack multiple decorators and see call order
- Use decorators with methods and simple class decorators

The first part reviews the basic pattern, then we get progressively fancier.

## 1. Basic decorator pattern

Reusable pattern:

```python
def wrapper(func):
    def inner(*args, **kwargs):
        # pre
        result = func(*args, **kwargs)
        # post
        return result
    return inner
```

In [1]:
def wrapper(func):
    def inner(*args, **kwargs):
        result = func(*args, **kwargs)
        return result
    return inner

We'll wrap a few different kinds of functions to show that `*args` / `**kwargs` makes it generic.

In [2]:
def add(a, b, c):
    return a + b + c

def greet(name):
    return f'Hello {name}!'

def join(data, *, item_sep=',', line_sep='\n'):
    return line_sep.join(
        [
            item_sep.join(str(item) for item in row) 
            for row in data
        ]
    )     

In [3]:
add(1, 2, 3)

6

In [4]:
greet('Python')

'Hello Python!'

In [5]:
join([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

'1,2,3\n4,5,6\n7,8,9'

In [6]:
add_wrapped = wrapper(add)
greet_wrapped = wrapper(greet)
join_wrapped = wrapper(join)

In [7]:
add_wrapped(1, 2, 3)

6

In [8]:
greet_wrapped('Python')

'Hello Python!'

In [9]:
join_wrapped([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

'1,2,3\n4,5,6\n7,8,9'

So far the wrapper is just a transparent pass-through. Let's make it actually do something useful.

## 2. A simple logging decorator

First a very simple one that prints to stdout:

In [10]:
def log(func):
    def inner(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__} called... result={result}')
        return result
    return inner

In [11]:
add_logged = log(add)
greet_logged = log(greet)
join_logged = log(join)

In [12]:
add_logged(1, 2, 3)

add called... result=6


6

In [13]:
greet_logged('Python')

greet called... result=Hello Python!


'Hello Python!'

We'd rather not rename everything to `*_logged`. Instead, we want `add` itself to refer to the wrapped version.

Pattern:

```python
def add(...):
    ...
add = log(add)
```

In [14]:
def log(func):
    def inner(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__} called... result={result}')
        return result
    return inner

def add(a, b, c):
    return a + b + c
add = log(add)

def greet(name):
    return f'Hello {name}!'
greet = log(greet)

def join(data, *, item_sep=',', line_sep='\n'):
    return line_sep.join([item_sep.join(str(item) for item in row) for row in data])     
join = log(join)

In [15]:
greet('Python')

greet called... result=Hello Python!


'Hello Python!'

That pattern is so common that Python has the `@decorator` syntax, which is just sugar for `func = decorator(func)`.

In [16]:
def log(func):
    def inner(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__} called... result={result}')
        return result
    return inner

@log
def add(a, b, c):
    return a + b + c

@log
def greet(name):
    return f'Hello {name}!'

@log
def join(data, *, item_sep=',', line_sep='\n'):
    return line_sep.join([item_sep.join(str(item) for item in row) for row in data])     

In [17]:
add(1, 2, 3)

add called... result=6


6

## 3. Using the `logging` module and timing

Instead of `print`, we'll use the standard `logging` module, and also measure execution time.

In [18]:
import logging

In [19]:
logging.basicConfig(
    format='%(asctime)s %(levelname)s: %(message)s',
    level=logging.DEBUG
)

In [20]:
logger = logging.getLogger('Custom Log')

In [21]:
logger.debug('Information message')

2025-11-13 09:51:32,472 DEBUG: Information message


In [22]:
logger.error('Some error happened')

2025-11-13 09:51:32,671 ERROR: Some error happened


In [23]:
logger.warning('Some warning')

2025-11-13 09:51:32,854 WARNING: Some warning


Now let's write a logging decorator that logs to this logger and tracks execution time.

In [24]:
from time import perf_counter

def log(func):
    def inner(*args, **kwargs):
        start = perf_counter()
        result = func(*args, **kwargs)
        end = perf_counter()
        logger.debug(f'called={func.__name__}, elapsed={end-start}')
        return result
    return inner

In [25]:
@log
def add(a, b, c):
    return a + b + c

@log
def greet(name):
    return f'Hello {name}!'

@log
def join(data, *, item_sep=',', line_sep='\n'):
    return line_sep.join([item_sep.join(str(item) for item in row) for row in data])     

In [26]:
add(10, 20, 30)

2025-11-13 09:51:33,600 DEBUG: called=add, elapsed=3.100140020251274e-06


60

In [27]:
join([range(10) for _ in range(10)])

2025-11-13 09:51:33,785 DEBUG: called=join, elapsed=0.0006941999308764935


'0,1,2,3,4,5,6,7,8,9\n0,1,2,3,4,5,6,7,8,9\n0,1,2,3,4,5,6,7,8,9\n0,1,2,3,4,5,6,7,8,9\n0,1,2,3,4,5,6,7,8,9\n0,1,2,3,4,5,6,7,8,9\n0,1,2,3,4,5,6,7,8,9\n0,1,2,3,4,5,6,7,8,9\n0,1,2,3,4,5,6,7,8,9\n0,1,2,3,4,5,6,7,8,9'

> Up to here: core decorator pattern + logging + timing.

Now we’ll go into **advanced** improvements: `functools.wraps`, parameterized decorators, stacking, and class usage.

## 4. Preserving metadata with `functools.wraps`

Right now, our decorated functions lose their original `__name__`, docstring, etc.

We can check:

In [28]:
def plain_func(x, y):
    """Add two numbers."""
    return x + y

@log
def decorated_func(x, y):
    """Add two numbers (decorated)."""
    return x + y

In [29]:
plain_func.__name__, decorated_func.__name__  # metadata lost on decorated_func

('plain_func', 'inner')

In [30]:
from functools import wraps

def log_wraps(func):
    @wraps(func)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = func(*args, **kwargs)
        end = perf_counter()
        logger.debug(f'called={func.__name__}, elapsed={end-start}')
        return result
    return inner

In [31]:
@log_wraps
def decorated_func2(x, y):
    """Add two numbers (decorated with wraps)."""
    return x + y

In [32]:
decorated_func2.__name__, decorated_func2.__doc__

('decorated_func2', 'Add two numbers (decorated with wraps).')

`@wraps` makes the wrapper look like the original function to tools like `help()`, debuggers, etc. In real code, you almost always want it.

## 5. Parameterized decorators (decorators with arguments)

Sometimes you want to configure the decorator:

- log level
- prefix name
- number of retries, etc.

This means your decorator itself must accept arguments and return an actual decorator.

In [33]:
def log_with_level(level=logging.DEBUG):
    """Decorator factory: returns a decorator that logs at `level`."""
    def decorator(func):
        @wraps(func)
        def inner(*args, **kwargs):
            start = perf_counter()
            result = func(*args, **kwargs)
            end = perf_counter()
            logger.log(level, f'called={func.__name__}, elapsed={end-start}')
            return result
        return inner
    return decorator

In [34]:
@log_with_level(logging.INFO)
def slow_add(a, b):
    """Pretend to be slow."""
    from time import sleep
    sleep(0.01)
    return a + b

@log_with_level(logging.WARNING)
def risky_div(a, b):
    return a / b

In [35]:
slow_add(1, 2)

2025-11-13 09:51:48,138 INFO: called=slow_add, elapsed=0.010265100048854947


3

In [36]:
risky_div(4, 2)

2025-11-13 09:51:48,324 WARNING: called=risky_div, elapsed=4.200031980872154e-06


2.0

General pattern for parameterized decorators:

```python
def deco_factory(arg1, arg2):
    def actual_decorator(func):
        @wraps(func)
        def inner(*a, **kw):
            # use arg1, arg2, func, a, kw
            return func(*a, **kw)
        return inner
    return actual_decorator
```

## 6. Stacking multiple decorators

You can layer decorators. Order matters:

```python
@d1
@d2
def f(...):
    ...
```

is equivalent to:

```python
f = d1(d2(f))
```

In [37]:
def debug_calls(tag):
    def decorator(func):
        @wraps(func)
        def inner(*args, **kwargs):
            logger.debug(f'[{tag}] entering {func.__name__} args={args}, kwargs={kwargs}')
            result = func(*args, **kwargs)
            logger.debug(f'[{tag}] exiting {func.__name__} result={result}')
            return result
        return inner
    return decorator

def timer(func):
    @wraps(func)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = func(*args, **kwargs)
        end = perf_counter()
        logger.info(f'{func.__name__} took {end-start:.6f}s')
        return result
    return inner

In [38]:
@timer
@debug_calls('MATH')
def mul_add(a, b, c):
    from time import sleep
    sleep(0.01)
    return a * b + c

In [39]:
mul_add(2, 3, 4)

2025-11-13 09:51:49,737 DEBUG: [MATH] entering mul_add args=(2, 3, 4), kwargs={}
2025-11-13 09:51:49,752 DEBUG: [MATH] exiting mul_add result=10
2025-11-13 09:51:49,754 INFO: mul_add took 0.017319s


10

- First `debug_calls('MATH')` runs (inner logs enter/exit)
- Then `timer` wraps *that* function and logs the duration

Changing the order changes which wrapper is outermost.

## 7. Decorators and methods

Decorators work fine on instance methods, class methods, and static methods. The first argument (`self` or `cls`) is just part of `*args`.

In [40]:
class Calculator:
    def __init__(self, factor):
        self.factor = factor

    @timer
    @debug_calls('CALC')
    def scale(self, x):
        return self.factor * x

    @classmethod
    @debug_calls('CLASS')
    def identity(cls, x):
        return x

    @staticmethod
    @debug_calls('STATIC')
    def cube(x):
        return x ** 3

In [41]:
calc = Calculator(5)
calc.scale(10)

2025-11-13 09:51:50,738 DEBUG: [CALC] entering scale args=(<__main__.Calculator object at 0x000002221AB296A0>, 10), kwargs={}
2025-11-13 09:51:50,740 DEBUG: [CALC] exiting scale result=50
2025-11-13 09:51:50,741 INFO: scale took 0.003115s


50

In [42]:
Calculator.identity(123)

2025-11-13 09:51:50,955 DEBUG: [CLASS] entering identity args=(<class '__main__.Calculator'>, 123), kwargs={}
2025-11-13 09:51:50,956 DEBUG: [CLASS] exiting identity result=123


123

In [43]:
Calculator.cube(3)

2025-11-13 09:51:51,146 DEBUG: [STATIC] entering cube args=(3,), kwargs={}
2025-11-13 09:51:51,147 DEBUG: [STATIC] exiting cube result=27


27

## 8. A simple class decorator

A **class decorator** takes a class object and returns a (maybe modified) class.

Example: decorate all public methods of a class with `debug_calls` automatically.

In [44]:
def debug_all_methods(tag):
    def class_decorator(cls):
        for name, attr in list(cls.__dict__.items()):
            if name.startswith('_'):
                continue  # skip private / dunder
            if callable(attr):
                decorated = debug_calls(tag)(attr)
                setattr(cls, name, decorated)
        return cls
    return class_decorator

In [45]:
@debug_all_methods('SERVICE')
class Service:
    def ping(self):
        return 'pong'

    def add(self, x, y):
        return x + y

In [46]:
svc = Service()
svc.ping()

2025-11-13 09:51:51,965 DEBUG: [SERVICE] entering ping args=(<__main__.Service object at 0x000002221AB29940>,), kwargs={}
2025-11-13 09:51:51,967 DEBUG: [SERVICE] exiting ping result=pong


'pong'

In [47]:
svc.add(2, 3)

2025-11-13 09:51:52,198 DEBUG: [SERVICE] entering add args=(<__main__.Service object at 0x000002221AB29940>, 2, 3), kwargs={}
2025-11-13 09:51:52,203 DEBUG: [SERVICE] exiting add result=5


5

## 9. Takeaways

- A decorator is just a callable that takes a function and returns a new callable.
- Use `*args` / `**kwargs` to make wrappers generic.
- Always use `functools.wraps` to preserve metadata.
- For configurable behavior, use a **decorator factory** (a function that returns a decorator).
- Decorators stack; remember that the topmost decorator is applied last.
- Decorators work with functions, methods, classmethods, staticmethods, and even whole classes.

In real-world code you’ll often use decorators from libraries (`@lru_cache`, `@dataclass`, `@staticmethod`, `@property`, etc.), and understanding this pattern makes them far less "magical".